In [2]:

                                        # ------------------------------#
                                              # DATA PRE-PROCESSING #
                                        #-------------------------------#




import os
import pandas as pd
import glob

import warnings
warnings.simplefilter("ignore", UserWarning)


#Merging all patients Diabietes Dataset into one file. 
#Combined all patient files into one.
#This code loads all CSV files and combines them into a single consolidated dataset.


# Folder containing all raw patient files
raw_folder = r'C:\PYTHON HACK'
cleaned_folder = r'C:\PYTHON HACK\Cleaned_data'
os.makedirs(cleaned_folder, exist_ok=True)

# Loop through all CSVs
for file in os.listdir(raw_folder):
    if file.lower().endswith('.csv'):
        file_path = os.path.join(raw_folder, file)
        print(f"🧼 Cleaning {file}")

        try:
            # Load with flexible delimiter handling
            df = pd.read_csv(file_path, sep=None, engine='python', on_bad_lines='skip')

            # Standardize column names
            df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

            # Parse time column if present
            time_cols = [col for col in df.columns if 'time' in col]
            if time_cols:
                df[time_cols[0]] = pd.to_datetime(df[time_cols[0]], errors='coerce')
                df.dropna(subset=[time_cols[0]], inplace=True)

            # Drop fully empty rows
            df.dropna(how='all', inplace=True)

            # Save cleaned file
            cleaned_path = os.path.join(cleaned_folder, f'cleaned_{file}')
            df.to_csv(cleaned_path, index=False)
            print(f"✅ Saved: {cleaned_path}")

        except Exception as e:
            print(f"❌ Failed to clean {file}: {e}")


🧼 Cleaning HUPA0001P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0001P.csv
🧼 Cleaning HUPA0002P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0002P.csv
🧼 Cleaning HUPA0003P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0003P.csv
🧼 Cleaning HUPA0004P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0004P.csv
🧼 Cleaning HUPA0005P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0005P.csv
🧼 Cleaning HUPA0006P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0006P.csv
🧼 Cleaning HUPA0007P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0007P.csv
🧼 Cleaning HUPA0009P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0009P.csv
🧼 Cleaning HUPA0010P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0010P.csv
🧼 Cleaning HUPA0011P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0011P.csv
🧼 Cleaning HUPA0014P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0014P.csv
🧼 Cleaning HUPA0015P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA00

In [3]:


# Path to cleaned files
cleaned_folder = r'C:\PYTHON HACK\Cleaned_data'

# Grab all patient files (excluding demographics)
patient_files = glob.glob(os.path.join(cleaned_folder, 'cleaned_HUPA*.csv'))

# Merge all into one DataFrame
merged_df = pd.concat([pd.read_csv(f) for f in patient_files], ignore_index=True)

# Optional: Add filename as patient ID if not already present
merged_df['source_file'] = [os.path.basename(f) for f in patient_files for _ in range(len(pd.read_csv(f)))]

# Save merged file
merged_path = os.path.join(cleaned_folder, 'merged_patients.csv')
merged_df.to_csv(merged_path, index=False)

print(f"✅ Merged file saved at: {merged_path}")


✅ Merged file saved at: C:\PYTHON HACK\Cleaned_data\merged_patients.csv


In [4]:
# regex extraction
merged_df['patient_id'] = merged_df['source_file'].str.extract(r'(HUPA\d{4}P)')


In [6]:

# Read and merge all CSVs into one DataFrame
all_files = glob.glob(r'C:\PYTHON HACK\Cleaned_data\cleaned_HUPA*.csv')
merged_df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
merged_df.to_csv(r'C:\PYTHON HACK\Cleaned_data\merged_patients.csv', index=False)


In [9]:

# Add source file name to each row
merged_df['source_file'] = [os.path.basename(f) for f in all_files for _ in range(len(pd.read_csv(f)))]

# Extract patient_id from filename (e.g., HUPA0001P)
merged_df['patient_id'] = merged_df['source_file'].str.extract(r'(HUPA\d{4}P)')


In [11]:
print("Matched rows:", final_df['age'].notna().sum())
print("Unmatched rows:", final_df['age'].isna().sum())


Matched rows: 309392
Unmatched rows: 0


In [5]:

# A copy of the merged dataset is created to preserve the original.
# The time column is converted to date, time ,hour and days of week
# This allows us to easily to analysis Glucose spikes by hour, Activity trends by weekday,Insulin delivery timing


# Convert 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Create new columns
df['date'] = df['time'].dt.date
df['clock_time'] = df['time'].dt.time
df['hour'] = df['time'].dt.hour
df['day_of_week'] = df['time'].dt.day_name()

# Save the updated dataset to Excel
df.to_excel(r'C:\PYTHON HACK\Cleaned_data\final_dataset_with_time_features.xlsx', index=False)

print("✅ Time features added and saved to Excel!")


✅ Time features added and saved to Excel!


In [6]:
# Glucose Cleaning, Flagging, and Conversion

# Step 1: Fill missing glucose values with the median
df['glucose'] = df['glucose'].fillna(df['glucose'].median())

# Step 2: Flag glucose levels
df['glucose_flag'] = pd.cut(
    df['glucose'],
    bins=[0, 70, 180, 300],
    labels=['Low', 'Normal', 'High'],
    include_lowest=True
)

# Step 3: Convert mg/dL to mmol/L (1 mmol/L = 18 mg/dL)
df['glucose_mmol'] = df['glucose'] / 18

# Optional: Round mmol values for cleaner display
df['glucose_mmol'] = df['glucose_mmol'].round(2)

# Save updated file
df.to_excel(r'C:\PYTHON HACK\Cleaned_data\final_dataset_with_glucose.xlsx', index=False)

print("✅ Glucose features added and saved!")


✅ Glucose features added and saved!


In [7]:
# Normalize calories to a 0–1 scale
df['calories_norm'] = (df['calories'] - df['calories'].min()) / (df['calories'].max() - df['calories'].min())

# Flag days with high calorie burn
df['high_burn'] = df['calories'] > 2500


In [8]:
# Apply rolling average with a window of 5 readings
df['hr_rolling'] = df['heart_rate'].rolling(window=5, min_periods=1).mean()


In [9]:
# Rolling Average for Heart Rate
#Heart rate data can be noisy — especially if it's collected frequently (e.g., every few seconds or minutes).
#A rolling average helps: Smooth out short-term fluctuations
#Highlight trends over time (e.g., resting vs active periods)
#Make visualizations cleaner and more interpretable
#This is especially useful in dashboards or time-series plots where spikes can distract from the overall pattern.

# Step 1: Convert 'time' column to datetime if not already
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Step 2: Sort by time to ensure rolling window follows chronological order
df = df.sort_values('time')

# Step 3: Apply rolling average with a window of 5 readings
df['hr_rolling'] = df['heart_rate'].rolling(window=5, min_periods=1).mean()

# Optional: Round for cleaner display
df['hr_rolling'] = df['hr_rolling'].round(2)

# Save the updated file
df.to_excel(r'C:\PYTHON HACK\Cleaned_data\final_dataset_with_hr_smooth.xlsx', index=False)

print("✅ Heart rate smoothed and saved!")


✅ Heart rate smoothed and saved!


In [14]:
print(df.columns.tolist())

['patient_id', 'time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input', 'age', 'gender', 'race', 'average_sleep_duration_(hrs)', 'sleep_quality_(1-10)', '%_with_sleep_disturbances', 'hr_rolling']


In [15]:
# conversion of patient_id into string type 
df['patient_id'] = df['patient_id'].astype('string')

In [17]:
#conversion of numeric into INT(glucose,steps,heart_rate)
df = df.astype({'glucose': int, 'steps':int,'heart_rate': int})
print(df.info)

<bound method DataFrame.info of        patient_id                time  glucose  calories  heart_rate  steps  \
0       HUPA0001P 2018-06-13 18:40:00      332   6.35950          82     34   
1       HUPA0001P 2018-06-13 18:45:00      326   7.72800          83      0   
2       HUPA0001P 2018-06-13 18:50:00      330   4.74950          80      0   
3       HUPA0001P 2018-06-13 18:55:00      324   6.35950          89     20   
4       HUPA0001P 2018-06-13 19:00:00      306   5.15200          92      0   
...           ...                 ...      ...       ...         ...    ...   
309387  HUPA0028P 2022-05-18 11:55:00      109  10.79280         104      0   
309388  HUPA0028P 2022-05-18 12:00:00      114   9.80346         103      0   
309389  HUPA0028P 2022-05-18 12:05:00      118   5.66622          95      0   
309390  HUPA0028P 2022-05-18 12:10:00      123   5.57628          91      0   
309391  HUPA0028P 2022-05-18 12:15:00      128   5.57628          99      0   

        basal_rate 

In [19]:
# Round float columns to two decimal places: 
The columns calories, basal_rate, bolus_volume_delivered, and carb_input are rounded to two decimal places using .round(). 
This standardizes the numeric precision, making the data easier to read and interpret, while maintaining sufficient accuracy for analysis.
df = df.round({'calories': 2, 'basal_rate': 2,'bolus_volume_delivered': 2,'carb_input': 2 })
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 309392 entries, 0 to 309391
Data columns (total 16 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   patient_id                    309392 non-null  string        
 1   time                          309392 non-null  datetime64[ns]
 2   glucose                       309392 non-null  int32         
 3   calories                      309392 non-null  float64       
 4   heart_rate                    309392 non-null  int32         
 5   steps                         309392 non-null  int32         
 6   basal_rate                    309392 non-null  float64       
 7   bolus_volume_delivered        309392 non-null  float64       
 8   carb_input                    309392 non-null  float64       
 9   age                           309392 non-null  int64         
 10  gender                        309392 non-null  object        
 11  race              

In [23]:
#Checking for null (missing) values.It is an essential step in data cleaning because missing data can lead to errors, inaccurate calculations, or biased analysis
#Identifying null values allows you to decide how to handle these values.
df.isnull().sum()


patient_id                      0
time                            0
glucose                         0
calories                        0
heart_rate                      0
steps                           0
basal_rate                      0
bolus_volume_delivered          0
carb_input                      0
age                             0
gender                          0
race                            0
average_sleep_duration_(hrs)    0
sleep_quality_(1-10)            0
%_with_sleep_disturbances       0
hr_rolling                      0
dtype: int64

In [24]:
#Checking for the duplicate rows
#Duplicate rows can lead to biased results, incorrect statistics, and misleading insights.
#Checking for duplicates ensures data integrity by identifying and removing repeated records, so the analysis reflects accurate and unique


# Returns True for rows that are duplicates
duplicates = df.duplicated()

# Count total duplicates
print("Number of duplicate rows:", duplicates.sum())

Number of duplicate rows: 0


In [27]:
# Always check the outliers finding the outliers in the dataset is.
#Outliers are data points that differ significantly from the majority of observations.
#They can skew results, distort averages, or affect machine learning models.
#Identifying outliers helps in deciding whether to investigate, correct, or
#remove them to improve data quality and reliability of analysis.

clip_ranges = {
    "glucose": (40, 400), # mg/dL
    "heart_rate": (40, 200), # bpm
    "steps": (0, 1000), # steps in 5 min
    "calories": (0, 1000),# kcal in 5 min
    "basal_rate": (0, 20), # units/hour
    "bolus_volume_delivered": (0, 25), # units per 5 min
    "carb_input": (0, 300), # grams in 5 min
}

 for col, (low, high) in clip_ranges.items():

    if col in df.columns:
        below = (df[col] < low).sum() if low is not None else 0
        above = (df[col] > high).sum() if high is not None else 0
        print(f"{col}: below={below}, above={above}")

glucose: below=0, above=104
heart_rate: below=12, above=0
steps: below=0, above=0
calories: below=0, above=0
basal_rate: below=0, above=0
bolus_volume_delivered: below=4, above=0
carb_input: below=0, above=0


In [30]:

#The cleaned and processed DataFrame is exported to a CSV file named Cleaned_T1DM_patient)demographics.csv 
#The 'index=False' parameter ensures that row numbers are not included in the saved file.
#This creates a ready-to-use dataset for further analysis or modeling.

df_demographics = pd.read_csv(r'C:\PYTHON HACK\Cleaned_data\cleaned_T1DM_patient_demographics.csv')
df_demographics.to_csv("cleaned_T1DM_patient_demographics.csv", index=False)
print(df_demographics.head())


  patient_id  age  gender             race  average_sleep_duration_(hrs)  \
0  HUPA0001P   34    Male            Other                           6.3   
1  HUPA0002P   49    Male         Hispanic                           6.6   
2  HUPA0003P   64    Male            Black                           5.3   
3  HUPA0004P   34  Female  Native American                           5.2   
4  HUPA0005P   49    Male  Native American                           5.8   

   sleep_quality_(1-10)  %_with_sleep_disturbances  
0                   4.5                         80  
1                   4.4                         40  
2                   5.2                         70  
3                   6.9                         60  
4                   7.9                         30  
